In [1]:
# Global Variables Set by User **change these values before running script**

# URL of the portal with the template content
PROJECT_URL = "https://ps-cc.maps.arcgis.com"

# Portal Username for Log-on (needs to be admin)
PROJECT_USERNAME = "gbushong_PS_CC"

# URL of the portal that will be customized and have users added
DELIVERY_URL = "https://pdo-scripts.maps.arcgis.com"

# Delivery username for log-on (needs to be admin)
DELIVERY_USERNAME = "pdo_admin"

# delivery org UX Component locations - local full path
FOLDER = "/Users/grac9792/OneDrive - Esri/pdo-scripts/Sample_Config" 

# project groups to share with *ALL* delivery users
GROUP_IDS = ["5e831faa6e574e46ae2e019d0e897a4f"]

# featured group to display on delivery home page #aec-test-group
FEATURED_GROUP_ID = "5e831faa6e574e46ae2e019d0e897a4f"

# filenames of UI components and user file, which are located in FOLDER
THUMBNAIL_FILENAME = "thumbnail.png"
FOOTER_FILENAME = "footer.txt"
BACKGROUND_FILENAME = "background.png"
BANNER_FILENAME = "banner.png"
DESCRIPTION_FILENAME = "description.txt"
USERS_FILENAME = "users.csv"

In [2]:
# import libraries
from arcgis.gis import GIS
from arcgis.gis import Group
from arcgis.gis import UserManager
import os
import csv

In [3]:
# get UI component filepaths
THUMBNAIL = os.path.join(FOLDER, THUMBNAIL_FILENAME)
FOOTER = os.path.join(FOLDER, FOOTER_FILENAME)
BACKGROUND = os.path.join(FOLDER, BACKGROUND_FILENAME)
BANNER = os.path.join(FOLDER, BANNER_FILENAME)
DESCRIPTION = os.path.join(FOLDER, DESCRIPTION_FILENAME)
USERS = os.path.join(FOLDER, USERS_FILENAME)
USER_FIELDS = ["Email", "First Name", "Last Name", "Username", "Password", "Role", "User Type", "groups"]

In [4]:
# connect to project and delivery orgs
print("Connecting to {} with username {}...".format(PROJECT_URL, PROJECT_USERNAME))
project = GIS(PROJECT_URL, PROJECT_USERNAME)

print("Connecting to {} with username {}...".format(DELIVERY_URL, DELIVERY_USERNAME))
delivery = GIS(DELIVERY_URL, DELIVERY_USERNAME)

Connecting to https://ps-cc.maps.arcgis.com with username gbushong_PS_CC...
Enter password: ········
Connecting to https://pdo-scripts.maps.arcgis.com with username pdo_admin...
Enter password: ········


In [5]:
# Creation of functions to be used later
def optional_args_handler(key, user, default=None):
    """Sets optional user creation arguments appropriately"""
    try:
        var = None
        if key in user:
            var = user[key]
        else:
            if default:
                var = default
        return var
    except Exception as e:
        print("Optional args handler failed with args {}, {}, and {}: {}".format(key, user, default, e))

def add_user(user, origin, target, groups=[]):
    """Add user to the gis and to specified groups
    * Abstraction for creating from dict such as with csv

    args:
    user -- a dictionary containing user fields, see fields:
        http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.UserManager.create
    origin -- the project org, where users will be invited to groups
    target -- the delivery org, where users are added
    groups -- (optional) destination groups, compliments those in dict
    """
    try:
        print("INFO: Creating user {}".format(user["Username"]))
        role = optional_args_handler('Role', user, 'viewer') # default is viewer
        user_type = optional_args_handler('User Type', user, 'viewer') # default is viewer
        
        # create array of groups for user
        user_groups = []
        for grp in groups:
            user_groups.append(grp)

        group_str = user.pop('groups', None)
        if group_str:
            group_list = group_str.split(",")
            for g in group_list:
                try:
                    group_search = Group(origin, g)
                    if group_search.title:
                        user_groups.append(group_search)
                except Exception as e:
                    print("ERR: Could not find group id {}".format(g))
                    continue
        print("User will be invited to join: {}".format([g.title for g in user_groups]))
        
        result = target.users.create(username=user['Username'], password=user['Password'], firstname=user['First Name'], 
                                  lastname=user['Last Name'], email=user['Email'], role=role, user_type=user_type)

        # create users returns None if it was unsuccessful
        if not result:
            print("Did not create user: Check username {}".format(user['Username']))
            return

        # Invite user to groups
        print("Inviting to groups")
        for g in user_groups:
            try:
                res = target.groups.get(g.groupid).invite_users([user['Username']])
                if res == True:
                    print("Invited user {} to group {}".format(user['Username'], g.title))
                else: # res == False
                    print("Failed to invite user {} to group {}: {}".format(user['Username'], g.title, res))
            except Exception as e:
                print("ERR: Could not add user {} to group {}: {}".format(user['Username'], g, e))
                continue
        return result
    except Exception as e:
        print("ERR: Could not create User {}: Add User encountered error {}".format(user['Username'], e))

def add_users_csv(csv_file, origin, target, groups=None):
    """Add users from csv to gis

    args:
    csv_file -- path to csv with users to create
    origin -- project organization
    target -- delivery organization where users are added
    groups -- (optional) destination groups, compliments those in csv (default [])
    """
    try:
        results = []
        with open(csv_file, 'r') as users_csv:
            users = csv.DictReader(users_csv, fieldnames=USER_FIELDS)
            next(users, None) # skip the header
            for user in users:
                result = add_user(user, origin, target, groups=groups)
                results.append(result)
        return results
    except Exception as e:
        print("Add users csv failed with args {}, {}, {}, and {}: {}".format(csv_file, origin, target, groups, e))

In [ ]:
# fetch groups where *ALL* users will be added
share_groups = tuple(Group(project, g) for g in GROUP_IDS)
print(share_groups)

# add users to the delivery org and invite them to groups in project org
add_users_csv(USERS, project, delivery, groups=share_groups)

In [7]:
# # Python based UI customization is compatible with legacy
# # ArcGIS Online Home pages only at time of writing. 

# # customize UI by setting background, banner, logo, etc.
# ux = delivery.admin.ux
# ux.set_background(BACKGROUND)
# ux.set_banner(BANNER)
# ux.set_logo(THUMBNAIL)

# # read description text file and set portal description
# with open(DESCRIPTION, 'r') as f:
#     description = f.read()
#     ux.description = description
    
# # find and set featured group
# if FEATURED_GROUP_ID:
#     featured_group = Group(delivery, FEATURED_GROUP_ID)
#     ux.featured_content = {'group': featured_group}